In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import sys

dir2 = os.path.abspath("")
dir1 = os.path.dirname(dir2)
if dir1 not in sys.path:
    sys.path.append(dir1)

os.chdir("..")

In [2]:
import yaml
from src.datasets.datasets import CPDDatasets
from src.ensembles.ensembles import EnsembleCPDModel, DistanceEnsembleCPDModel
from src.baselines.prediction_utils import get_repr_learning_output, post_process_output
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload

comet_ml is installed but `COMET_API_KEY` is not set.


# BCE/InDiD HAR

In [3]:
model_type = "seq2seq"

experiments_name = "human_activity"

# read config file
path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, "r") as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["loss_type"] = "bce"
args_config["num_workers"] = 2

# prepare datasets
train_dataset, test_dataset = CPDDatasets(experiments_name).get_dataset_()
test_dataloader = DataLoader(
    test_dataset, batch_size=1, shuffle=False
)

In [4]:
ens_num = 1

path_to_models_folder = f"saved_models/{args_config['loss_type']}/{experiments_name}/full_sample/ens_{ens_num}"
# path_to_models_folder = f"saved_models/tscp/human_activity/window_{args_config['model']['window']}/ens_{ens_num}"

ens_bce = EnsembleCPDModel(args_config, n_models=10, boot_sample_size=None)
ens_bce.load_models_list(path_to_models_folder)

single_model_bce = ens_bce.models_list[0]

ens_bce_wass = DistanceEnsembleCPDModel(ens_bce, window_size=2, distance="wasserstein_1d")
ens_bce_mmd = DistanceEnsembleCPDModel(ens_bce, window_size=2, distance="mmd")

In [5]:
test_seq, _ = next(iter(test_dataloader))
test_seq = test_seq.detach().cpu()
test_seq.shape

torch.Size([1, 20, 28])

In [6]:
%%timeit
outs = single_model_bce(test_seq)

294 µs ± 18.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%%timeit
outs = ens_bce.predict(test_seq)

4.53 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit
outs = ens_bce_wass.predict(test_seq)

19.7 ms ± 75.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
outs = ens_bce_mmd.predict(test_seq)

16.6 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


# TS-CP HAR

In [18]:
model_type = "tscp"
experiments_name = "human_activity"

# read config file
path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, "r") as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["num_workers"] = 2

In [27]:
ens_num = 1

path_to_models_folder = f"saved_models/tscp/human_activity/window_{args_config['model']['window']}/ens_{ens_num}"

ens_tscp = EnsembleCPDModel(args_config, n_models=10, boot_sample_size=None)
ens_tscp.load_models_list(path_to_models_folder)

single_model_tscp = ens_tscp.models_list[0]

ens_tscp_wass = DistanceEnsembleCPDModel(ens_tscp, window_size=2, distance="wasserstein_1d")
ens_tscp_mmd = DistanceEnsembleCPDModel(ens_tscp, window_size=2, distance="mmd")

In [22]:
%%timeit
outs = get_repr_learning_output(
    model=single_model_tscp,
    batch=test_seq,
    window_1=single_model_tscp.window_1,
    window_2=single_model_tscp.window_2,
    step=args_config["predictions"]["step"],
    max_pool=False
)
outs = post_process_output(outs, alpha=args_config["predictions"]["alpha"])

9.46 ms ± 73.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
%%timeit
outs = ens_tscp.predict(
    test_seq, scale=args_config["predictions"]["scale"], step=args_config["predictions"]["step"], alpha=args_config["predictions"]["alpha"]
)

102 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
%%timeit
outs = ens_tscp_wass.predict(
    test_seq, scale=args_config["predictions"]["scale"], step=args_config["predictions"]["step"], alpha=args_config["predictions"]["alpha"]
)

104 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%%timeit
outs = ens_tscp_mmd.predict(
    test_seq, scale=args_config["predictions"]["scale"], step=args_config["predictions"]["step"], alpha=args_config["predictions"]["alpha"]
)

107 ms ± 701 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# TS2Vec-SN HAR

In [33]:
model_type = "ts2vec"
experiments_name = "human_activity"

# read config file
path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, "r") as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["num_workers"] = 2

In [34]:
ens_num = 1

path_to_models_folder = f"saved_models/ts2vec/human_activity/ens_{ens_num}"

ens_ts2vec = EnsembleCPDModel(args_config, n_models=10, boot_sample_size=None)
ens_ts2vec.load_models_list(path_to_models_folder)

single_model_ts2vec = ens_ts2vec.models_list[0]

ens_ts2vec_wass = DistanceEnsembleCPDModel(ens_ts2vec, window_size=2, distance="wasserstein_1d")
ens_ts2vec_mmd = DistanceEnsembleCPDModel(ens_ts2vec, window_size=2, distance="mmd")

In [36]:
%%timeit
outs = get_repr_learning_output(
    model=single_model_ts2vec,
    batch=test_seq,
    window_1=single_model_tscp.window_1,
    window_2=single_model_tscp.window_2,
    step=args_config["predictions"]["step"],
    max_pool=True
)
outs = post_process_output(outs, alpha=args_config["predictions"]["alpha"])

16.8 ms ± 762 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
%%timeit
outs = ens_ts2vec.predict(
    test_seq, scale=args_config["predictions"]["scale"], step=args_config["predictions"]["step"], alpha=args_config["predictions"]["alpha"]
)

176 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
outs = ens_ts2vec_wass.predict(
    test_seq, scale=args_config["predictions"]["scale"], step=args_config["predictions"]["step"], alpha=args_config["predictions"]["alpha"]
)

181 ms ± 8.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%%timeit
outs = ens_ts2vec_mmd.predict(
    test_seq, scale=args_config["predictions"]["scale"], step=args_config["predictions"]["step"], alpha=args_config["predictions"]["alpha"]
)

181 ms ± 7.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
